In [90]:
import os
import pandas as pd
import torch
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [91]:
csv_file ='./mergedlbls.csv'
labels = pd.read_csv(csv_file)
labels.tail()

,im_name,class_id,Xc,Yc,w,h
3565,RainyDay1/01738.jpg,0,1785,604,265,79
3566,RainyDay1/01766.jpg,1,1354,608,81,46
3567,RainyDay1/01767.jpg,1,1429,602,111,53
3568,RainyDay1/01768.jpg,1,1543,592,139,66
3569,RainyDay1/01769.jpg,1,1740,584,205,81


In [93]:
classes= {'car':0,'van':1,'truck':2,'bus':3,'motorcycle':4} # object classes
i2c = lambda i : [k for k,v in classes.items() if v ==i] # convert values to classes
i2c(0)

['car']

In [94]:
class dataset(Dataset):
    def __init__(self,csv_file, root_dir, transform=None):
        
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
    

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        frame_name = os.path.join(self.root_dir,self.labels.iloc[idx,0])# finding the frame path
        condition = re.findall(r'\w+[0-9]+/',self.labels.iloc[idx,0])
        condition = re.sub('\d+/', '', condition[0])
        frame = io.imread(frame_name) # reading frame
        cls_id,x,y,w,h = self.labels.iloc[idx,1:]
        bb = (x,y,w,h) # bounding box coordinates             
        
        sample = {'frame':frame,'class id':cls_id,'condition':condition,'bounding box':bb}
        
        if self.transform:
            image = self.transform(image)
            
        return sample

In [95]:
# initiate dataset
dataset = dataset(csv_file = './mergedlbls.csv', root_dir = 'Frames/')

In [96]:
dataset[0]

{'frame': array([[[ 21,  19,  24],
         [ 21,  19,  24],
         [ 21,  19,  22],
         ...,
         [255, 252, 255],
         [255, 252, 255],
         [255, 252, 255]],
 
        [[ 21,  19,  24],
         [ 21,  19,  24],
         [ 21,  19,  22],
         ...,
         [255, 252, 255],
         [255, 252, 255],
         [255, 252, 255]],
 
        [[ 21,  19,  22],
         [ 21,  19,  22],
         [ 21,  19,  22],
         ...,
         [255, 252, 255],
         [255, 252, 255],
         [255, 252, 255]],
 
        ...,
 
        [[  0,   0,   0],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [ 55,  52,  59],
         [ 55,  52,  59],
         [ 55,  52,  59]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  1,   1,   1],
         ...,
         [ 44,  41,  48],
         [ 44,  41,  48],
         [ 44,  41,  48]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [ 39,  36,